### Beispiel-Code für Indexierung und Querying, Implementierung einer semantischen Suchfunktion mit Milvus 

##### Voraussetzungen: 
1. `venv` aktivieren!
2. Instalation der Bibliothek: `pip install pymilvus sentence-transformers`
3. Docker Desktop installieren
4. In Projekt Ordner:
   
      `mkdir docker-milvus`

      `cd docker-milvus` 

      Docker-Konfiguration herunterladen:

      `wget https://github.com/milvus-io/milvus/releases/download/v2.4.6/milvus-standalone-docker-compose.yml -O docker-compose.yml` 

      Falls `wget` nicht verfügbar, dann einfach im Browser öffnen.

      Server lokal via Docker starten:

      `docker-compose up -d`

In [7]:
texts = [
    'Wie kann ich mein Passwort Passwort zurücksetzen?',
    'Wie ändere ich mein Passwort?',
    'Passwort vergessen - wie kann ich es zurücksetzen?',
    'Wie kann ich mein Login-Passwort erneuern?',
    'Wo finde ich meine Bestellhistorie?',
    'Wie kann ich meine bisherigen Bestellungen einsehen?',
    'Wo sehe ich meine vergangenen Bestellungen?',
    'Wo kann ich meine Bestellungen überprüfen?',
    'Bestellhistorie abrufen wie geht das?',
    'Wie kann ich meine Lieferadresse ändern?',
    'Lieferadresse aktualisieren wie geht das?',
    'Wie ändere ich meine Versandadresse?',
    'Kann ich meine Adresse nachträglich ändern?',
    'Adresse für Lieferung ändern möglich?',
    'Wie kontaktiere ich den Kundendienst?',
    'Wie erreiche ich den Support?',
    'Kundendienst kontaktieren wie?',
    'Wo kann ich den Kundenservice erreichen?',
    'Wie bekomme ich Hilfe vom Support-Team?',
    'Welche Zahlungsmethoden werden akzeptiert?',
    'Welche Bezahlmöglichkeiten gibt es?',
    'Wie kann ich bezahlen?',
    'Akzeptierte Zahlungsmethoden - Übersicht',
    'Welche Zahlungsarten stehen zur Verfügung?',
    'Wie kann ich meine Bestellung stornieren?',
    'Bestellung rückgängig machen wie?',
    'Wie annulliere ich eine Bestellung?',
    'Kann ich meine Bestellung noch stornieren?',
    'Stornierung einer Bestellung Anleitung',
    'Wie lange dauert der Versand?',
    'Versanddauer - wie lange dauert es?',
    'Wann wird meine Bestellung geliefert?',
    'Lieferzeitraum - wie lang ist er?',
    'Wie schnell kommt meine Bestellung an?',
    'Kann ich Artikel nach der Bestellung noch ändern?',
    'Kann ich meine Bestellung nachträglich bearbeiten?',
    'Artikel in einer bereits getätigten Bestellung ändern - geht das?',
    'Bestellung nachträglich ändern - möglich?',
    'Kann ich Produkte nach der Bestellung austauschen?'
]

In [8]:
# Beispiel: Milvus Vektor-Datenbank

from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from sentence_transformers import SentenceTransformer
import numpy as np

# Verbindung zu lokal Milvus herstellen
connections.connect("default", host="127.0.0.1", port="19530")

# modell für embeddings laden
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Collection-Schema definieren
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=4096)
]
schema = CollectionSchema(fields, description="Demo Collection für semantische Suche")

# Neue Collection erstellen. Falls die Collection schon existiert, löschen
if utility.has_collection("demo_collection"):
    utility.drop_collection("demo_collection")

collection = Collection(name="demo_collection", schema=schema)

# Index erstellen (z. B. HNSW)
index_params = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 16, "efConstruction": 200}
}
collection.create_index(field_name="embedding", index_params=index_params)

# Embeddings erzeugen
embeddings = model.encode(texts).astype(np.float32).tolist()
# Daten einfügen
collection.insert([embeddings, texts])  # IDs werden automatisch vergeben

collection.load()  # Daten in Arbeitsspeicher laden

# Semantische Suche durchführen
query_text = "Bestellung history"
query_emb = model.encode([query_text]).astype(np.float32).tolist()

results = collection.search(
    data=query_emb,
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"ef": 50}},
    limit=3,
    output_fields=["text"]
)

# Testen
for hits in results:
    for hit in hits:
        print(f"Score: {hit.score:.3f} | Text: {hit.entity.get('text')}")


Score: 0.597 | Text: Bestellhistorie abrufen wie geht das?
Score: 0.575 | Text: Wie schnell kommt meine Bestellung an?
Score: 0.571 | Text: Wo finde ich meine Bestellhistorie?
